\%cd '/content/drive/MyDrive/Job Assignment'






---



In [ ]:
# Commented out IPython magic to ensure Python compatibility.
from google.colab import drive
drive.mount('/content/drive')

# Change directory to your working directory
%cd '/content/drive/MyDrive/Job Assignment'

# Import necessary packages
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
import re

# Read the URL file into the pandas object
df = pd.read_excel('Input.xlsx')

# Create the directory if it doesn't exist
directory_path = '/content/drive/MyDrive/Job Assignment/TitleText'
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

# Loop through each row in the DataFrame
for index, row in df.iterrows():
    url = row['URL']
    url_id = row['URL_ID']

    # Make a request to the URL
    header = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"}
    try:
        response = requests.get(url, headers=header)
    except:
        print("Can't get response of {}".format(url_id))
        continue

    # Create a BeautifulSoup object
    try:
        soup = BeautifulSoup(response.content, 'html.parser')
    except:
        print("Can't get page of {}".format(url_id))
        continue

    # Find title
    try:
        title = soup.find('h1').get_text()
    except:
        print("Can't get title of {}".format(url_id))
        continue

    # Find text
    article = ""
    try:
        for p in soup.find_all('p'):
            article += p.get_text()
    except:
        print("Can't get text of {}".format(url_id))
        continue

    # Write title and text to the file
    file_name = f'/content/drive/MyDrive/Job Assignment/TitleText/{url_id}.txt'
    with open(file_name, 'w') as file:
        file.write(title + '\n' + article)

print("Text extraction completed.")


Mounted at /content/drive
/content/drive/MyDrive/Job Assignment


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Can't get title of blackassign0036
Can't get title of blackassign0049
Text extraction completed.


In [ ]:
import os
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords as nltk_stopwords
import nltk

nltk.download('punkt')
nltk.download('stopwords')

# Define directories
text_dir = "/content/drive/MyDrive/Job Assignment/TitleText"
stopwords_dir = "/content/drive/MyDrive/Job Assignment/StopWords"
sentiment_dir = "/content/drive/MyDrive/Job Assignment/MasterDictionary"

# Ensure the directory for saving text files exists
if not os.path.exists(text_dir):
    os.makedirs(text_dir)

# Load input data
input_df = pd.read_excel('Input.xlsx')

# Define a function to extract article title and text
def extract_article(url):
    header = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"}
    response = requests.get(url, headers=header)
    soup = BeautifulSoup(response.content, 'html.parser')

    title = soup.find('h1').get_text() if soup.find('h1') else "Title not found"
    article = " ".join([p.get_text() for p in soup.find_all('p')])
    if not article:
        article = "Text not found"

    return title, article

# Extract articles and save to text files
for index, row in input_df.iterrows():
    url = row['URL']
    url_id = row['URL_ID']

    try:
        title, article = extract_article(url)
        with open(os.path.join(text_dir, f'{url_id}.txt'), 'w') as file:
            file.write(f"{title}\n{article}")
    except Exception as e:
        print(f"Failed to extract {url}: {e}")

# Load stopwords
stop_words = set()
for file in os.listdir(stopwords_dir):
    with open(os.path.join(stopwords_dir, file), 'r', encoding='ISO-8859-1') as f:
        stop_words.update(f.read().splitlines())

# Load sentiment words
pos_words = set()
neg_words = set()
for file in os.listdir(sentiment_dir):
    with open(os.path.join(sentiment_dir, file), 'r', encoding='ISO-8859-1') as f:
        words = f.read().splitlines()
        if 'positive' in file:
            pos_words.update(words)
        elif 'negative' in file:
            neg_words.update(words)

# Define a function to analyze text and compute metrics
def analyze_text(file_path):
    with open(file_path, 'r') as file:
        text = file.read()

    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words and word.isalpha()]

    positive_words = [word for word in filtered_words if word.lower() in pos_words]
    negative_words = [word for word in filtered_words if word.lower() in neg_words]

    positive_score = len(positive_words)
    negative_score = len(negative_words)
    polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
    subjectivity_score = (positive_score + negative_score) / (len(filtered_words) + 0.000001)

    sentences = re.split(r'[.!?]+', text)
    num_sentences = len(sentences)
    avg_sentence_length = len(filtered_words) / num_sentences if num_sentences > 0 else 0

    complex_words = [word for word in filtered_words if len(re.findall(r'[aeiouy]', word.lower())) > 2]
    percentage_complex_words = len(complex_words) / len(filtered_words) if len(filtered_words) > 0 else 0
    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)

    word_lengths = [len(word) for word in filtered_words]
    avg_word_length = sum(word_lengths) / len(word_lengths) if word_lengths else 0

    syllable_count = sum(len(re.findall(r'[aeiouy]', word.lower())) for word in filtered_words)
    syllables_per_word = syllable_count / len(filtered_words) if len(filtered_words) > 0 else 0

    personal_pronouns = len(re.findall(r'\b(I|we|my|ours|us)\b', text, re.I))

    metrics = {
        'POSITIVE SCORE': positive_score,
        'NEGATIVE SCORE': negative_score,
        'POLARITY SCORE': polarity_score,
        'SUBJECTIVITY SCORE': subjectivity_score,
        'AVG SENTENCE LENGTH': avg_sentence_length,
        'PERCENTAGE OF COMPLEX WORDS': percentage_complex_words,
        'FOG INDEX': fog_index,
        'AVG NUMBER OF WORDS PER SENTENCE': avg_sentence_length,
        'COMPLEX WORD COUNT': len(complex_words),
        'WORD COUNT': len(filtered_words),
        'SYLLABLE PER WORD': syllables_per_word,
        'PERSONAL PRONOUNS': personal_pronouns,
        'AVG WORD LENGTH': avg_word_length
    }

    return metrics

# Analyze each text file and collect results
results = []
for index, row in input_df.iterrows():
    url_id = row['URL_ID']
    file_path = os.path.join(text_dir, f'{url_id}.txt')
    if os.path.exists(file_path):
        metrics = analyze_text(file_path)
        results.append({**row, **metrics})

# Convert results to DataFrame
output_df = pd.DataFrame(results)

# Save the results to CSV
output_df.to_csv('Output_Data.csv', index=False)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
